In [25]:
import os
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import cv2


In [7]:
def split_and_augment_dataset(input_folder, train_folder, test_folder, test_ratio=0.2, augment_count=5):
    """
    Splits the dataset into training and test sets, augments the images, and saves them to respective folders.
    
    Parameters:
    - input_folder (str): Path to the dataset folder containing subdirectories for each class.
    - train_folder (str): Path to the folder where the augmented training dataset will be saved.
    - test_folder (str): Path to the folder where the augmented test dataset will be saved.
    - test_ratio (float): Proportion of images to use as the test set.
    - augment_count (int): Number of augmented images to generate per original image.
    """
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    # Set up data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for class_name in os.listdir(input_folder):
        class_folder = os.path.join(input_folder, class_name)
        if os.path.isdir(class_folder):
            images = os.listdir(class_folder)
            random.shuffle(images)
            
            num_test_images = int(len(images) * test_ratio)
            test_images = images[:num_test_images]
            train_images = images[num_test_images:]

            # Create class directories in the train and test folders
            train_class_folder = os.path.join(train_folder, class_name)
            test_class_folder = os.path.join(test_folder, class_name)
            os.makedirs(train_class_folder, exist_ok=True)
            os.makedirs(test_class_folder, exist_ok=True)

            # Augment and save test images
            for img_name in test_images:
                img_path = os.path.join(class_folder, img_name)
                img = load_img(img_path)
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)

                # Save original image in the test folder
                img.save(os.path.join(test_class_folder, img_name))

                # Generate augmented images
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=test_class_folder, save_prefix='aug', save_format='jpeg'):
                    i += 1
                    if i >= augment_count:
                        break

            # Augment and save train images
            for img_name in train_images:
                img_path = os.path.join(class_folder, img_name)
                img = load_img(img_path)
                x = img_to_array(img)
                x = x.reshape((1,) + x.shape)

                # Save original image in the train folder
                img.save(os.path.join(train_class_folder, img_name))

                # Generate augmented images
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=train_class_folder, save_prefix='aug', save_format='jpeg'):
                    i += 1
                    if i >= augment_count:
                        break

    print("Augmented images saved to train and test directories.")

# Define paths
input_folder = 'dataset_frames'  # Path to your dataset folder
train_folder = 'train_dataset'  # Path to the training dataset folder
test_folder = 'test_dataset'  # Path to the test dataset folder

# Call the function to split and augment the dataset
split_and_augment_dataset(input_folder, train_folder, test_folder, test_ratio=0.2, augment_count=5)


Augmented images saved to train and test directories.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load data from the augmented datasets
train_generator = train_datagen.flow_from_directory(
    train_folder,  # Path to augmented training dataset directory
    target_size=(224, 224),  # Resize images
    batch_size=32,  # Batch size for training
    class_mode='categorical'  # Use categorical labels
)

test_generator = test_datagen.flow_from_directory(
    test_folder,  # Path to augmented test dataset directory
    target_size=(224, 224),  # Resize images
    batch_size=32,  # Batch size for testing
    class_mode='categorical',  # Use categorical labels
    shuffle=False  # No need to shuffle for evaluation
)


Found 4558 images belonging to 8 classes.
Found 1138 images belonging to 8 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Define a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=35,  # Start with a reasonable number of epochs
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    callbacks=[early_stopping]
)


Epoch 1/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 432s 3s/step - accuracy: 0.7356 - loss: 6.0054 - val_accuracy: 0.0902 - val_loss: 188.0824
Epoch 2/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.3856 - val_accuracy: 0.0000e+00 - val_loss: 52.5232
Epoch 3/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 408s 3s/step - accuracy: 0.9346 - loss: 0.6885 - val_accuracy: 0.1580 - val_loss: 46.9089
Epoch 4/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.2783 - val_accuracy: 0.8333 - val_loss: 0.7178
Epoch 5/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 409s 3s/step - accuracy: 0.9474 - loss: 0.4748 - val_accuracy: 0.3795 - val_loss: 29.4146
Epoch 6/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.8667 - val_accuracy: 0.5556 - val_loss: 2.6055
Epoch 7/35
142/142 ━━━━━━━━━━━━━━━━━━━━ 407s 3s/step - accuracy: 0.9566 - loss: 0.6302 - val_accuracy: 0.6357 - val_loss: 23.6598


In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 448ms/step - accuracy: 0.2261 - loss: 35.6704
Test Loss: 47.9523, Test Accuracy: 0.1571


In [12]:
# Savemodel
model.save('model2_2.h5')


In [23]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image_class(model, img_path, class_labels, confidence_threshold=0.6):
    # Load the image
    img = image.load_img(img_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Expand dimensions to match the input shape of the model
    img_array = np.expand_dims(img_array, axis=0)

    # Rescale pixel values
    img_array /= 255.0

    # Make a prediction
    predictions = model.predict(img_array)

    # Get the maximum predicted probability
    max_probability = np.max(predictions)

    # Get the index of the highest probability class
    predicted_class_index = np.argmax(predictions, axis=1)[0]

    # Get the class label
    predicted_class_label = class_labels[predicted_class_index]

    # Check if the prediction confidence is above the threshold
    if max_probability < confidence_threshold:
        predicted_class_label = "unknown"

    print(f"Predicted class: {predicted_class_label}, Confidence: {max_probability:.2f}")

# Define class labels from the training data
class_labels = list(train_generator.class_indices.keys())

# Test with a new image
img_path = 'testdata\Screenshot 2024-08-05 191556.png'  # Replace with the path to your image
predict_image_class(model, img_path, class_labels, confidence_threshold=0.6)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted class: nagasai, Confidence: 0.99


In [28]:
def detect_faces(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the pre-trained Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    return faces

def recognize_faces(model, image_path, class_labels, confidence_threshold=0.6):
    img = cv2.imread(image_path)
    faces = detect_faces(image_path)

    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (224, 224))
        face_array = image.img_to_array(face_img)
        face_array = np.expand_dims(face_array, axis=0) / 255.0

        # Predict class
        predictions = model.predict(face_array)
        max_probability = np.max(predictions)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class_label = class_labels[predicted_class_index]

        # Check confidence
        if max_probability < confidence_threshold:
            predicted_class_label = "unknown"

        # Draw rectangle and label on the image
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(img, predicted_class_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the image
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Define class labels from the training data
class_labels = list(train_generator.class_indices.keys())

# Recognize faces in an image
image_path = 'testdata\Screenshot 2024-08-05 191435.png'
recognize_faces(model, image_path, class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
